<a href="https://colab.research.google.com/github/TanjilaPathan30/comp6651-project/blob/main/small_tutorial_on_optimization_part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading Iris dataset

In [ ]:
import torch
from sklearn import datasets
import matplotlib.pyplot as plt
import numpy as np


import matplotlib.pyplot as plt

In [ ]:
# Load Iris dataset
iris = datasets.load_iris()
X = iris.data
y = iris.target

# Filter for binary classification (only class 0 and class 1)
binary_class_indices = y <= 1
X = X[binary_class_indices]
y = y[binary_class_indices]

# Convert to torch tensors
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.int64)

# Spliting the dataset

In [ ]:
from sklearn.model_selection import train_test_split

seed = 15
# Split the dataset
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state= seed)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=seed)

print(f'Train size: {len(X_train)}, Validation size: {len(X_val)}, Test size: {len(X_test)}')

Train size: 60, Validation size: 20, Test size: 20


# Define the model

In [ ]:
# Initialize weights and bias
weights = torch.randn(X_train.shape[1], requires_grad=True)
bias = torch.randn(1, requires_grad=True)

# Sigmoid function
def sigmoid(x):
    return 1 / (1 + torch.exp(-x))

# Logistic regression model
def model(X):
    return sigmoid(X @ weights + bias)

# Train the model

In [ ]:
def accuracy(y_true, y_pred):
    predicted = (y_pred > 0.5).float()
    return (predicted == y_true).float().mean()

# Hyperparameters
learning_rate = 0.01
epochs = 1000

# Training loop
for epoch in range(epochs):
    # Forward pass on training set
    y_train_pred = model(X_train).squeeze()

    # Binary cross-entropy loss
    loss = -(y_train * torch.log(y_train_pred) + (1 - y_train) * torch.log(1 - y_train_pred)).mean()

    # Backward pass
    loss.backward()

    # Update weights and bias manually
    with torch.no_grad():
        weights -= learning_rate * weights.grad
        bias -= learning_rate * bias.grad

    grad_norm = torch.norm(weights.grad)
    # Zero gradients after updating
    weights.grad.zero_()
    bias.grad.zero_()

    # Validation evaluation
    with torch.no_grad():
        y_val_pred = model(X_val).squeeze()
        val_loss = -(y_val * torch.log(y_val_pred) + (1 - y_val) * torch.log(1 - y_val_pred)).mean()
        val_acc = accuracy(y_val, y_val_pred)

    # Print progress every 100 epochs
    if epoch % 100 == 0:
        print(f'Epoch {epoch}, gradient norm: {grad_norm.item():.4f}')
        print(f'Epoch {epoch}, Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}, Val Accuracy: {val_acc.item():.4f}')

Epoch 0, gradient norm: 0.8232
Epoch 0, Loss: 0.7219, Val Loss: 0.8088, Val Accuracy: 0.4500
Epoch 100, gradient norm: 0.4322
Epoch 100, Loss: 0.4226, Val Loss: 0.4501, Val Accuracy: 1.0000
Epoch 200, gradient norm: 0.3080
Epoch 200, Loss: 0.2870, Val Loss: 0.2997, Val Accuracy: 1.0000
Epoch 300, gradient norm: 0.2343
Epoch 300, Loss: 0.2138, Val Loss: 0.2194, Val Accuracy: 1.0000
Epoch 400, gradient norm: 0.1873
Epoch 400, Loss: 0.1694, Val Loss: 0.1712, Val Accuracy: 1.0000
Epoch 500, gradient norm: 0.1554
Epoch 500, Loss: 0.1399, Val Loss: 0.1395, Val Accuracy: 1.0000
Epoch 600, gradient norm: 0.1324
Epoch 600, Loss: 0.1191, Val Loss: 0.1173, Val Accuracy: 1.0000
Epoch 700, gradient norm: 0.1153
Epoch 700, Loss: 0.1037, Val Loss: 0.1009, Val Accuracy: 1.0000
Epoch 800, gradient norm: 0.1020
Epoch 800, Loss: 0.0918, Val Loss: 0.0884, Val Accuracy: 1.0000
Epoch 900, gradient norm: 0.0914
Epoch 900, Loss: 0.0824, Val Loss: 0.0786, Val Accuracy: 1.0000


# Evaluate on the test set

In [ ]:
# Evaluate on test set
with torch.no_grad():
    y_test_pred = model(X_test).squeeze()
    test_acc = accuracy(y_test, y_test_pred)
    print(f'Test Accuracy: {test_acc.item():.4f}')

Test Accuracy: 1.0000
